#  Introductions to constrainat-based modeling using cobrapy

## Part 2: Flux Balance Analysis

### Instructor:
* Miguel Ponce de León from (Barcelona Supercomputing Center)
* Contact: miguel.ponce@bsc.es

11 December, 2020

# Part 2: Genome-scale modelling.

In this part we are gonna use a genome-scale metabolic model of Escherichia coli named iJO1366
The file has already been stored in the data folder and its path is data/iJO1366.xml

Alternatively, you can also access it here:
- [http://bigg.ucsd.edu/models/iJO1366](http://bigg.ucsd.edu/models/iJO1366)

to download the model and to see other metadata (citation, description, etc)

## Part 2.1: Studying the model.

### Read the SBML model

First we need to import the function read_sbml_model from the cobra.io modules

In [ ]:
import cobra
from cobra.io import read_sbml_model

# State the path to the file iJO1366.xml
sbml_fname = './data/iJO1366.xml'

# Read the model
model = read_sbml_model(sbml_fname)

## Exercise 2.1: Inspecting the model's numbers

How many metabolites genes and reactions are contained in the model?

In [ ]:
## TODO
## Write your code below


In [ ]:
# Inspecting the genes
# 1. Access a particular reaction:
#    * You can do it directly with: gene = model.genes.b0720
#    * Or you can use the function get_by_id: gene = model.genes.get_by_id('b0720')

b0720 = model.genes.b0720

b0722 = model.genes.get_by_id('b0722')

# 6. Inspect the reaction by printing:
#    1. gene.name
#    2. gene.id
#    3. gene.reactions

### Inspecting the systems' boundaries

* see the exchange fluxes
* see the objective function (the reaction set to be optimized)

Use print(model.summary())

You can also find the objective function using the following filtering technique:
* [r for r in model.reactions if r.objective_coefficient == 1]
* the reaction id of the biomass is Ec_biomass_iJO1366_WT_53p95M
and the exchange fluxes can be accessed using:
* model.boundary

In [ ]:
## TODO
## Write your code below


### Running a Flux Balance Analysis (FBA).

Documentation: [https://cobrapy.readthedocs.io/en/latest/simulating.html](https://cobrapy.readthedocs.io/en/latest/simulating.html)

By default, the model boundary condition (growth medium) is M9 aerobic (glucose minimal)

1.  Check the medium by inspecting the lower_bound of the following reactions:
  * EX\_glc\_e\_.lower_bound
  * EX\_o2\_e\_.lower_bound
2.  Optimize biomass using: 
  * solution = model.optimize()
  
3.  Inspect the solution as we did previously in Part 1.2 Optimization.



In [ ]:
solution = model.optimize()

print("Objective value: %.2f\n" % solution.objective_value)
print("Status: %s\n" % solution.status)

print("Fluxes:\n")
print(solution.fluxes)

# Converting the solution into a pandas dataframe
df = solution.to_frame()
# Saving the solution into tab-separed-value (tsv) format (plain text)
df.to_csv("out/iJO1366_fba.tsv", sep="\t")

Inspect the flux value of the following reactions
* The glucose consumption: EX_glc_e_
* The oxygen consumption: EX_o2_e_
* The growth rate: Ec_biomass_iJO1366_WT_53p95M

HINT 1: use the solution object -> solution.fluxes.reaction_id <br>
HINT 2: use model.summary()

In [ ]:
## TODO
## Write your code below


## Exercise 2.2: 

1. Change the oxygen exchange lower bound to zero to simulate anaerobic growth.
2. Optimize the model
3. What is the maximal growth rate in anaerobic conditions
4. what are the main three secretion products?

In [ ]:
## TODO
## Write your code below


## Exercise 2.3: 

1. Set the oxygen exchange lower bound to -20
2. Set the glucose exchange flux (EX_glc_D_e_ lower bound to 0)
3. Set the glucose exchange flux (EX_ac_e_ lower bound to -10)

What is the maximal growth rate using acetate as soley carbon source
what is the oxygen uptake rate?

In [ ]:
## TODO
## Write your code below


## Optional Visualizing flux distributions using Escher

[Escher documentation](https://escher.readthedocs.io/en/latest/)

Escher online WebApp: [https://escher.github.io/](https://escher.github.io/#/)

In [ ]:
import escher
from escher import Builder

# Lets crate a builder by passing our model as well a given map name to tell escher how to represent the network
# Check the escher web to see other maps https://escher.github.io/#/
builder = Builder(organism='Escherichia coli', map_name='iJO1366.Central metabolism')
builder.reaction_data = solution.fluxes
builder

In [ ]:
# Add the optimal flux distribution to our map builder
model.reactions.EX_o2_e_.lower_bound = -20
model.reactions.EX_glc_e_.lower_bound = -10
model.reactions.EX_ac_e_.lower_bound = 0


gene =  model.genes.b0720
with model:
    gene.knock_out()
    ko_solution = model.optimize()

builder.reaction_data = ko_solution.fluxes
builder